# Caption Generation from Images CNN & RNN キャプション生成

![title](https://images.unsplash.com/photo-1517924250218-eb05042519bc?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1052&q=80)

### Task
与えられた画像の説明文（キャプション）の作成を自動化するAI
### Architecture
CNN & LSTM  転移学習
### Framework
Keras ( TensorFlow backend )
### Reserch Paper
Where to put the Image in an Image Caption Generator (https://arxiv.org/abs/1703.09137)  
Oxford Visual Geometry Group, or VGG, model that won the ImageNet competition in 2014   
Very Deep Convolutional Networks for Large-Scale Visual Recognition (http://www.robots.ox.ac.uk/~vgg/research/very_deep/)
### Process
0. タスク定義
1. データ準備（Flickr 8k）
2. ニューラルネット構築（転移学習）
3. 学習&テスト
4. 評価&改善（BLEU scores）

![title](https://cdn-images-1.medium.com/max/800/1*E90mI7YT9F0J6b9EadxfzA.png)

## 1. Data Preparation

- 写真の内容を解釈するために、事前に訓練されたモデルを使用します。
- その後、これらの機能を後でロードして、データセット内の特定の写真の解釈としてモデルに入力できます。  
- VGGクラスを使用して、KerasにVGGモデルをロードできます。
- これは写真の分類を予測するために使用されるモデルなので、ロードされたモデルから最後のレイヤーを削除します。
- 画像の分類には興味がありませんが、分類が行われる直前の写真の内部表現には興味があります。
- これらは、モデルが写真から抽出した「特徴」です。
- Kerasはまた、ロードした写真をモデルに適したサイズに変形するためのツールも提供します（例：3チャンネル224 x 224ピクセルの画像）。
- 下はextract_features（）という名前の関数です。
- ディレクトリ名を指定すると、各写真をロードしてVGG用に準備し、VGGモデルから予測された特徴を収集します。

In [2]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Using TensorFlow backend.


### A) Prepare Text Data 画像から特徴量を生成

#### VGG16 is a CNN architecture named Visual Geometry Group from Oxford


##### 大量の画像データに対して、VGG16を用いた転移学習を実行する

- 転移学習：既存の学習済モデル（出力層以外の部分）を、重みデータは変更せずに特徴量抽出機として利用する
- ファインチューニング：既存の学習済モデル（出力層以外の部分）を、重みデータを一部再学習して特徴量抽出機として利用する
  
###### 転移学習の手順
1. 入力画像から、特徴量(ボトルネック特徴量)を抽出する
2. ボトルネック特徴量を用いて、クラス分類をする
  
　つまり転移学習では、VGG16など大規模なデータを用いて学習した強力なモデルを特徴抽出器として利用し（多数の対象を分類できる為、画像の特徴を捉えるのが非常に上手い）、任意のクラスの分類する為の特徴量の圧縮器として利用しています。

In [6]:
def extract_features(curr_dir):
    # CNNの一つであるVGG16モデルで転移学習を行う
    model = VGG16() 
    # モデルの再構築
    model.layers.pop()
    model = Model( inputs = model.inputs, outputs = model.layers[-1].output )
    # モデルの統合
    print(model.summary())
    
    features = dict()
    # 各画像から特徴を生成する
    for name in listdir(curr_dir):
        filename = curr_dir + "/" + name
        image = load_img( filename, target_size=(224, 224) )
        # 計算の高速化のために画像のピクセル数をndarrayに変換する
        image = img_to_array(image)
        # 画像データを4次元アレイに変換する
        image = image.reshape(( 1, image.shape[0], image.shape[1], image.shape[2] ))
        # VGG16モデルへ入力するため、画像データを最終処理
        image = preprocess_input(image)
        # 特徴量を生成 ( verbose: 0, 1または2．詳細表示モード．0とすると標準出力にログを出力しません )
        feature = model.predict( image, verbose = 0 )
        # 画像のIDを生成
        image_id = name.split(".")[0]
        features[image_id] = feature
        print( ">%s"%name )
    
    # 戻り値はkey=画像ID, value=画像の特徴量という辞書 
    return features

上記の関数にデータを流し込んで特徴量生成を実行する

In [ ]:
curr_dir = "/Users/akr712/Desktop/CaptionGeneration_CNNandLSTM_Keras/Flicker8k_Dataset"
features = extract_features(curr_dir)
print( "生成された特徴量の次元数: %d" % len(features) )
# Pickleモジュールはプログラムを実行し終えたあとも作成したオブジェクトを保存する機能を提供してくれる
dump( features, open("features.pkl", "wb") )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

>2596514158_c516e57974.jpg
>2772532341_c4597a94ed.jpg
>3150252702_828a570d46.jpg
>2883099128_0b056eed9e.jpg
>3611672054_45edd3e08f.jpg
>1321723162_9d4c78b8af.jpg
>557101732_32bbc47c12.jpg
>162743064_bb242faa31.jpg
>2130203183_49bae96b96.jpg
>3707738261_777075e885.jpg
>2528521798_fb689eba8d.jpg
>3320209694_db579cb607.jpg
>2729147877_c3ec3445bf.jpg
>3582685410_05315a15b8.jpg
>3730011219_588cdc7972.jpg
>3204922011_185e48949a.jpg
>3126795109_73920ed5dc.jpg
>2427558437_3e839056d7.jpg
>2599444370_9e40103027.jpg
>3107463441_7c68606450.jpg
>540721368_12ac732c6c.jpg
>2260369648_e21ae6494a.jpg
>3522076584_7c603d2ac5.jpg
>294098577_c10f32bcfa.jpg
>2356664078_4b1e6e465d.jpg
>2644920808_f5a214b744.jpg
>2208067635_39a03834ca.jpg
>2640153227_57cf1a3d92.jpg
>3477715432_79d82487bb.jpg
>3301021288_95935b7a74.jpg
>252124738_796599e94b.jpg
>3212085754_35fdc9ccaa.jpg
>3441399292_60c83bd5db.jpg
>2635164923_2a774f7854.jpg
>3184206563_5435f2b494.jpg
>2490863987_715383944a.jpg
>2410618963_fb78307d18.jpg
>34200

>3417672954_46b75dea8d.jpg
>2187720319_112d00f07d.jpg
>2715337869_e4fe36db50.jpg
>3482668767_66004ce736.jpg
>3399312265_9c74378692.jpg
>3565598162_56044bc2f7.jpg
>3159447439_c1496cbaea.jpg
>2881468095_d4ce8c0c52.jpg
>3175446111_681a89f873.jpg
>2646116932_232573f030.jpg
>2802337003_56e555cd30.jpg
>3467510271_0f57e52768.jpg
>2322593776_e6aaf69e80.jpg
>2999735171_87ca43c225.jpg
>2101457132_69c950bc45.jpg
>2735979477_eef7c680f9.jpg
>237277765_9e6fa5b99a.jpg
>488356951_b3b77ad832.jpg
>1355833561_9c43073eda.jpg
>3652572138_34d6b72999.jpg
>445861800_75fc6a8c16.jpg
>3119913014_688d4997d7.jpg
>3513362553_5fc5779e20.jpg
>2244171992_a4beb04d8e.jpg
>1895768965_43cd9d164f.jpg
>3430100177_5864bf1e73.jpg
>102455176_5f8ead62d5.jpg
>2924884815_63826aa60d.jpg
>3682428916_69ce66d375.jpg
>1622619190_d0b51aff28.jpg
>2752331711_cb18abba5a.jpg
>965444691_fe7e85bf0e.jpg
>2842439618_fb20fe2215.jpg
>2932740428_b15384f389.jpg
>361183669_52be9662b9.jpg
>108898978_7713be88fc.jpg
>359173181_a75c950aeb.jpg
>43493858

>140526326_da07305c1c.jpg
>3154709407_9b0778cbeb.jpg
>2070831523_5035d5537e.jpg
>498794783_cc2ac62b47.jpg
>3257207516_9d2bc0ea04.jpg
>652542470_60e858da64.jpg
>1745110280_0cbff5e273.jpg
>1474474514_b3eb492722.jpg
>854333409_38bc1da9dc.jpg
>3250593457_9049a73b61.jpg
>3638374272_444f5e0457.jpg
>2176364472_31fcd37531.jpg
>684375286_09cc1aa778.jpg
>3343311201_eeb1a39def.jpg
>468310111_d9396abcbd.jpg
>2544426580_317b1f1f73.jpg
>374103966_2987706be1.jpg
>3003691049_f4363c2d5c.jpg
>397286183_745abbf40d.jpg
>2256320794_0286c31bfa.jpg
>2598012140_832863fcb9.jpg
>2836360729_6500249fe6.jpg
>3687222696_85bf6f78f7.jpg
>297169473_d3974e0275.jpg
>224273695_0b517bd0eb.jpg
>2519812011_f85c3b5cb5.jpg
>2230067846_74046b89d3.jpg
>2607130765_97833d6ce1.jpg
>2081679622_6f1442367d.jpg
>460478198_83039f2593.jpg
>270816949_ffad112278.jpg
>3439128755_84409b8823.jpg
>3323528927_7b21081271.jpg
>3004823335_9b82cbd8a7.jpg
>324355356_859988a710.jpg
>3033825101_44a4390f16.jpg
>2873522522_829ea62491.jpg
>3087485737_cb

### B) Prepare Text Data テキストから特徴量を生成

In [ ]:
def load_text(filename):
    """
    function: ファイル内のテキストを抽出する
    input: テキストファイル
    output: テキスト
    """
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

filename = "Flickr8k_text/Flickr8k.token.txt"
description = load_text(filename)

<img src="inside_fole.png">

## A CNN LSTM Model